In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c961fa96e55c6b5c7c3e4dd0962b5c9de841d6141f2c20c3a623b182b8cba3a2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName('Fire DF').getOrCreate()
pyspark.sql.session.SparkSession

pyspark.sql.session.SparkSession

In [ ]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
data = spark.read.csv("drive/My Drive/Colab Notebooks/Data/sf-fire-calls.csv", header=True,inferSchema=True)

In [ ]:
data.show()
data.printSchema()

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

In [ ]:
data1 = data.select('IncidentNumber','CallType','CallDate','CallNumber','City','ZipCode', 'Neighborhood','Delay')
data1.show()

+--------------+----------------+----------+----------+----+-------+--------------------+---------+
|IncidentNumber|        CallType|  CallDate|CallNumber|City|ZipCode|        Neighborhood|    Delay|
+--------------+----------------+----------+----------+----+-------+--------------------+---------+
|       2003235|  Structure Fire|01/11/2002|  20110016|  SF|  94109|     Pacific Heights|     2.95|
|       2003241|Medical Incident|01/11/2002|  20110022|  SF|  94124|Bayview Hunters P...|      4.7|
|       2003242|Medical Incident|01/11/2002|  20110023|  SF|  94102|          Tenderloin|2.4333334|
|       2003250|    Vehicle Fire|01/11/2002|  20110032|  SF|  94110|      Bernal Heights|      1.5|
|       2003259|          Alarms|01/11/2002|  20110043|  SF|  94109|    Western Addition|3.4833333|
|       2003279|  Structure Fire|01/11/2002|  20110072|  SF|  94105|Financial Distric...|     1.75|
|       2003301|          Alarms|01/11/2002|  20110125|  SF|  94112|Oceanview/Merced/...|2.7166667|


In [ ]:
data2 = data1.withColumn('Date', to_date(col('CallDate'), 'mm/dd/yyyy')).drop('CallDate')
data2.show()

+--------------+----------------+----------+----+-------+--------------------+---------+----------+
|IncidentNumber|        CallType|CallNumber|City|ZipCode|        Neighborhood|    Delay|      Date|
+--------------+----------------+----------+----+-------+--------------------+---------+----------+
|       2003235|  Structure Fire|  20110016|  SF|  94109|     Pacific Heights|     2.95|2002-01-11|
|       2003241|Medical Incident|  20110022|  SF|  94124|Bayview Hunters P...|      4.7|2002-01-11|
|       2003242|Medical Incident|  20110023|  SF|  94102|          Tenderloin|2.4333334|2002-01-11|
|       2003250|    Vehicle Fire|  20110032|  SF|  94110|      Bernal Heights|      1.5|2002-01-11|
|       2003259|          Alarms|  20110043|  SF|  94109|    Western Addition|3.4833333|2002-01-11|
|       2003279|  Structure Fire|  20110072|  SF|  94105|Financial Distric...|     1.75|2002-01-11|
|       2003301|          Alarms|  20110125|  SF|  94112|Oceanview/Merced/...|2.7166667|2002-01-11|


In [ ]:
data3 = data2.withColumn('Year', year(col('Date')))\
.withColumn('Month', month(col('Date')))\
.withColumn('Week', weekofyear(col('Date')))

data3.show()

+--------------+----------------+----------+----+-------+--------------------+---------+----------+----+-----+----+
|IncidentNumber|        CallType|CallNumber|City|ZipCode|        Neighborhood|    Delay|      Date|Year|Month|Week|
+--------------+----------------+----------+----+-------+--------------------+---------+----------+----+-----+----+
|       2003235|  Structure Fire|  20110016|  SF|  94109|     Pacific Heights|     2.95|2002-01-11|2002|    1|   2|
|       2003241|Medical Incident|  20110022|  SF|  94124|Bayview Hunters P...|      4.7|2002-01-11|2002|    1|   2|
|       2003242|Medical Incident|  20110023|  SF|  94102|          Tenderloin|2.4333334|2002-01-11|2002|    1|   2|
|       2003250|    Vehicle Fire|  20110032|  SF|  94110|      Bernal Heights|      1.5|2002-01-11|2002|    1|   2|
|       2003259|          Alarms|  20110043|  SF|  94109|    Western Addition|3.4833333|2002-01-11|2002|    1|   2|
|       2003279|  Structure Fire|  20110072|  SF|  94105|Financial Distr

In [ ]:
def mapSeason(seas):
  if 2<seas<6:
    return 'Spring'
  elif 5<seas<9:
    return 'Summner'
  elif 8<seas<12:
    return 'Autumn'
  else:
    return 'Winter'
seasonUDF =  udf(mapSeason,StringType())
data4 = data3.withColumn('Season', seasonUDF(col('Month')))
data4.show()

+--------------+----------------+----------+----+-------+--------------------+---------+----------+----+-----+----+------+
|IncidentNumber|        CallType|CallNumber|City|ZipCode|        Neighborhood|    Delay|      Date|Year|Month|Week|Season|
+--------------+----------------+----------+----+-------+--------------------+---------+----------+----+-----+----+------+
|       2003235|  Structure Fire|  20110016|  SF|  94109|     Pacific Heights|     2.95|2002-01-11|2002|    1|   2|Winter|
|       2003241|Medical Incident|  20110022|  SF|  94124|Bayview Hunters P...|      4.7|2002-01-11|2002|    1|   2|Winter|
|       2003242|Medical Incident|  20110023|  SF|  94102|          Tenderloin|2.4333334|2002-01-11|2002|    1|   2|Winter|
|       2003250|    Vehicle Fire|  20110032|  SF|  94110|      Bernal Heights|      1.5|2002-01-11|2002|    1|   2|Winter|
|       2003259|          Alarms|  20110043|  SF|  94109|    Western Addition|3.4833333|2002-01-11|2002|    1|   2|Winter|
|       2003279|

In [ ]:
#1.Get yearly count of fire calls

getcount = data4.select('Year').groupBy('Year').count().orderBy('Year', ascending = False)
getcount.show()

+----+-----+
|Year|count|
+----+-----+
|2018|10136|
|2017|12135|
|2016|11609|
|2015|11458|
|2014|10775|
|2013|10020|
|2012| 9674|
|2011| 9735|
|2010| 9341|
|2009| 8789|
|2008| 8869|
|2007| 8255|
|2006| 8174|
|2005| 8282|
|2004| 8283|
|2003| 8499|
|2002| 8090|
|2001| 7713|
|2000| 5459|
+----+-----+



In [ ]:
#2.What were all the different types of fire calls in 2018

diffType = data4.select('CallType').where(col('Year')==2018).distinct()
diffType.show(truncate=False)

+-------------------------------+
|CallType                       |
+-------------------------------+
|Elevator / Escalator Rescue    |
|Alarms                         |
|Odor (Strange / Unknown)       |
|Citizen Assist / Service Call  |
|HazMat                         |
|Vehicle Fire                   |
|Other                          |
|Outside Fire                   |
|Traffic Collision              |
|Assist Police                  |
|Gas Leak (Natural and LP Gases)|
|Water Rescue                   |
|Electrical Hazard              |
|Structure Fire                 |
|Medical Incident               |
|Fuel Spill                     |
|Smoke Investigation (Outside)  |
|Train / Rail Incident          |
|Explosion                      |
|Suspicious Package             |
+-------------------------------+



In [ ]:
#3.Which week in the year in 2018 had the most fire calls
max_week = data4.select('Week').where(col('Year')==2018).groupBy('Week').count().orderBy('count',ascending = False)
max_week.show()

+----+-----+
|Week|count|
+----+-----+
|   1| 2584|
|   3| 2270|
|   2| 2249|
|   4| 2240|
|   5|  793|
+----+-----+



OR

In [ ]:
maxweek = data4.select('Week').where(col('Year')==2018).groupBy('Week').count()
maxweek.select('Week', 'count').filter(col('count')==maxweek.agg({'count':'max'}).collect()[0][0]).show()
#maxweek.select('Week', 'count').filter(col('count')==maxweek.agg({'count':'max'}).collect()[0][0]).collect()[0][1]

+----+-----+
|Week|count|
+----+-----+
|   1| 2584|
+----+-----+



In [ ]:
#4. Get monthly count of fire calls based on year

month_count = data4.select('Month').where(col('Year')==2017).groupBy('Month').count().orderBy('count', ascending = True)
month_count.show()

+-----+-----+
|Month|count|
+-----+-----+
|    1|12135|
+-----+-----+



In [ ]:
#5. Give monthlyreport of fire call types for selected year
selected_year = 2017
data5 = data4.filter(data4['Year'] == selected_year)
monthly_report = data5.groupBy('Year', 'Month', 'CallType' ).agg(count(col('CallType')).alias('count'))
monthly_report.show()

+----+-----+--------------------+-----+
|Year|Month|            CallType|count|
+----+-----+--------------------+-----+
|2017|    1|              Alarms| 1414|
|2017|    1|      Structure Fire| 1024|
|2017|    1|Confined Space / ...|    2|
|2017|    1|  Suspicious Package|    1|
|2017|    1|Odor (Strange / U...|   10|
|2017|    1|   Electrical Hazard|   38|
|2017|    1|Gas Leak (Natural...|   56|
|2017|    1|        Water Rescue|   84|
|2017|    1|          Fuel Spill|    9|
|2017|    1|        Vehicle Fire|   43|
|2017|    1|   High Angle Rescue|    2|
|2017|    1|   Traffic Collision|  531|
|2017|    1|Train / Rail Inci...|    4|
|2017|    1|    Medical Incident| 8330|
|2017|    1|        Outside Fire|  150|
|2017|    1|Smoke Investigati...|   29|
|2017|    1|Elevator / Escala...|   41|
|2017|    1|Citizen Assist / ...|  156|
|2017|    1|               Other|  189|
|2017|    1|              HazMat|   11|
+----+-----+--------------------+-----+
only showing top 20 rows



In [ ]:
#6. Give top five fire call types for every season of selected year
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
filtered_data = data4.filter(col('Year') == 2018)
partition_data = Window.partitionBy('Season').orderBy(col('count').desc())
ranked_data = filtered_data.groupBy('Season', 'CallType').count().withColumn("rank", rank().over(partition_data))
top_season = ranked_data.filter(col('rank') <= 5)
top_season.show()

+------+-----------------+-----+----+
|Season|         CallType|count|rank|
+------+-----------------+-----+----+
|Winter| Medical Incident| 7004|   1|
|Winter|           Alarms| 1144|   2|
|Winter|   Structure Fire|  906|   3|
|Winter|Traffic Collision|  433|   4|
|Winter|     Outside Fire|  153|   5|
+------+-----------------+-----+----+



In [ ]:
#8. What months within the year 2018 saw the highest number of fire calls
sel_year = data4.filter(col('Year') == 2018)
monthly_counts = filtered_data.groupBy('Month').count()
max_count = monthly_counts.agg({"count": "max"}).collect()[0][0]
top_months = monthly_counts.filter(col("count") == max_count)
top_months.show()

+-----+-----+
|Month|count|
+-----+-----+
|    1|10136|
+-----+-----+



In [ ]:
#10. Find out average delay in response for each call type
average_delay = data4.groupBy('CallType').agg(avg('Delay').alias('AverageDelay'))
average_delay.show()

+--------------------+------------------+
|            CallType|      AverageDelay|
+--------------------+------------------+
|Elevator / Escala...| 4.337821933487857|
|         Marine Fire| 6.928571314285715|
|  Aircraft Emergency|3.7731481500000004|
|Confined Space / ...| 6.915384576923078|
|      Administrative|12.261111333333332|
|              Alarms|3.5427290545084076|
|Odor (Strange / U...| 4.947959182000003|
|Citizen Assist / ...| 5.473342576604596|
|              HazMat| 7.527016126612902|
|Watercraft in Dis...| 6.886904817857142|
|           Explosion| 4.110674168539326|
|           Oil Spill| 4.977777761904762|
|        Vehicle Fire|3.9039227134074928|
|  Suspicious Package|6.5766667199999995|
|Extrication / Ent...| 4.391666678571427|
|               Other| 5.505155432421978|
|        Outside Fire| 4.181948425367714|
|   Traffic Collision| 3.789132088873234|
|       Assist Police|26.981903994285716|
|Gas Leak (Natural...|  4.58339877840314|
+--------------------+------------

In [ ]:
#11. Find which calltype has maximum average delay time

average_delay = data4.groupBy('CallType').agg(avg('Delay').alias('AverageDelay'))
max_average_delay = average_delay.orderBy(col('AverageDelay').desc()).first()
print("Call Type with Maximum Average Delay:")
print("Call Type:", max_average_delay['CallType'])
print("Average Delay:", max_average_delay['AverageDelay'])

Call Type with Maximum Average Delay:
Call Type: Mutual Aid / Assist Outside Agency
Average Delay: 38.41666631111111


In [ ]:
#12. Which neighborhood in San Francisco generated the most fire calls in 2018?

neighborhood_call_counts = data4.filter((col('Year') == 2018) & (col('Neighborhood').isNotNull())) \
    .groupBy('Neighborhood').count()
neighborhood_call_counts = neighborhood_call_counts.orderBy(col('count').desc()).first()
print("Neighborhood:",neighborhood_call_counts['Neighborhood'])
print("Number of Fire Calls:", neighborhood_call_counts['count'])

Neighborhood: Tenderloin
Number of Fire Calls: 1393


In [ ]:
#13. Which neighborhoods had the worst response times to fire calls in 2018?

filtered_data = data4.filter((col('Year') == 2018) & (col('Delay').isNotNull()))
neighbor_response_time = filtered_data.groupBy('Neighborhood').agg(avg('Delay').alias('AverageResponseTime'))
worst_response_time_neighbor = neighbor_response_time.orderBy(col('AverageResponseTime').desc())
worst_response_time_neighbor.show()

+--------------------+-------------------+
|        Neighborhood|AverageResponseTime|
+--------------------+-------------------+
|           Chinatown| 6.1903140979057625|
|            Presidio|  5.829227041449275|
|     Treasure Island|  5.453703712499999|
|        McLaren Park|  4.744047642857142|
|Bayview Hunters P...| 4.6205619568773955|
|    Presidio Heights|  4.594131472394366|
|        Inner Sunset|  4.438095199935065|
|      Inner Richmond|  4.364728682713178|
|Financial Distric...|  4.344084618290155|
|      Haight Ashbury|  4.266428599285713|
|            Seacliff|  4.261111146666667|
|  West of Twin Peaks|  4.190952390857144|
|        Potrero Hill|   4.19055555742857|
|     Pacific Heights|  4.180453718900521|
|          Tenderloin|   4.10151951659727|
|Oceanview/Merced/...|  3.947242180719425|
|           Excelsior| 3.9363993797169807|
|         North Beach| 3.8892496403463204|
|           Lakeshore|  3.881551365094342|
|         Mission Bay| 3.8548689521910124|
+----------

In [ ]:
#15. For each year find out which city has more calltype

calltype_by_city_year = data4.groupBy('Year', 'CallType', 'City').count()
window_spec = Window.partitionBy('Year', 'CallType').orderBy(col('count').desc())
max_calltype_by_city_year = calltype_by_city_year.withColumn('rank', rank().over(window_spec))
max_calls_by_city_year = max_calltype_by_city_year.filter(col('rank') == 1)
max_calls_by_city_year.show()

+----+--------------------+----+-----+----+
|Year|            CallType|City|count|rank|
+----+--------------------+----+-----+----+
|2000|  Aircraft Emergency| SFO|    3|   1|
|2000|              Alarms|  SF|  620|   1|
|2000|Citizen Assist / ...|  SF|  124|   1|
|2000|   Electrical Hazard|  SF|    6|   1|
|2000|Elevator / Escala...|  SF|   15|   1|
|2000|          Fuel Spill|  SF|   12|   1|
|2000|Gas Leak (Natural...|  SF|   11|   1|
|2000|Industrial Accidents|  SF|    4|   1|
|2000|    Medical Incident|  SF| 3399|   1|
|2000|Odor (Strange / U...|  SF|   19|   1|
|2000|           Oil Spill|  SF|    5|   1|
|2000|               Other|  SF|   99|   1|
|2000|        Outside Fire|  SF|   67|   1|
|2000|Smoke Investigati...|  SF|   12|   1|
|2000|      Structure Fire|  SF| 1002|   1|
|2000|  Suspicious Package|  SF|    1|   1|
|2000|Train / Rail Inci...|  SF|    2|   1|
|2000|        Vehicle Fire|  SF|   30|   1|
|2000|        Water Rescue|  SF|    7|   1|
|2001|  Aircraft Emergency| SFO|

In [ ]:
#16. For every year find count of calltypes  for 5 cities which has more calls

calltype_by_city_year = data4.groupBy('Year', 'City', 'CallType').count()
window_spec = Window.partitionBy('Year').orderBy(col('count').desc())
rankedcities_by_year = calltype_by_city_year.withColumn('rank', row_number().over(window_spec))
top5_cities = rankedcities_by_year.filter(col('rank') <= 5)
result = top5_cities.groupBy('Year', 'CallType').count()
result.show()

+----+--------------------+-----+
|Year|            CallType|count|
+----+--------------------+-----+
|2000|    Medical Incident|    1|
|2000|      Structure Fire|    1|
|2000|              Alarms|    1|
|2000|Citizen Assist / ...|    1|
|2000|               Other|    1|
|2001|    Medical Incident|    1|
|2001|      Structure Fire|    1|
|2001|              Alarms|    1|
|2001|               Other|    1|
|2001|Citizen Assist / ...|    1|
|2002|    Medical Incident|    1|
|2002|      Structure Fire|    1|
|2002|              Alarms|    1|
|2002|               Other|    1|
|2002|Citizen Assist / ...|    1|
|2003|    Medical Incident|    1|
|2003|      Structure Fire|    1|
|2003|              Alarms|    1|
|2003|   Traffic Collision|    1|
|2003|Citizen Assist / ...|    1|
+----+--------------------+-----+
only showing top 20 rows

